<a href="https://colab.research.google.com/github/nvwa0318/projects-Moonic-for-music-clustering/blob/main/movies_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import streamlit as st
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

ModuleNotFoundError: ignored

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 25.1 MB/s 
     |████████████████████████████████| 181 kB 55.1 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 235 kB 50.5 MB/s 
     |████████████████████████████████| 164 kB 45.5 MB/s 
     |████████████████████████████████| 4.3 MB 47.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 132 kB 53.7 MB/s 
     |████████████████████████████████| 793 kB 48.5 MB/s 
     |████████████████████████████████| 423 kB 53.7 MB/s 
     |████████████████████████████████| 132 kB 48.2 MB/s 
     |████████████████████████████████| 381 kB 53.1 MB/s 
     |████████████████████████████████| 51 kB 8.5 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=2a3f9ef53cd83a904bccd064eaa5d1c760bd4873988bd382f71f86f5e6b

In [ ]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

numpy==1.21.6
pandas==1.3.5
streamlit==1.12.0


In [ ]:
#movies = pd.read_csv('C:/Users/chicmachina/Desktop/WBS/recommender/ml-latest-small/movies.csv')
#ratings = pd.read_csv('C:/Users/chicmachina/Desktop/WBS/recommender/ml-latest-small/ratings.csv')
#links = pd.read_csv('C:/Users/chicmachina/Desktop/WBS/recommender/ml-latest-small/links.csv')

#tags = pd.read_csv('C:/Users/chicmachina/Desktop/WBS/recommender/ml-latest-small/tags.csv')

# rating_final.csv
url = 'https://drive.google.com/file/d/1e-gzJJsEX5lSKMWdnNbH7FfWejzHZE8d/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
tags = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1wYwW-mAqi5H1SIqpRlYTt7jrwHnJBYHl/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1rPrRdbUtCJGvqlDwPrkA7nhhzkND_LrF/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies = pd.read_csv(path)

# 'geoplaces2.csv'
url = 'https://drive.google.com/file/d/1-RyhCPBSrN-4rj3zKXhm6U5zQxVP5iUM/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
links = pd.read_csv(path) # change encoding to 'mbcs' in Windows


## Data preparation

In [ ]:
genres = movies.pop('genres').str.split('|', expand=True)

,0,1,2,3,4,5,6,7,8,9
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,Comedy,None,None,None,None,None,None,None,None,None


In [ ]:
genres = genres.stack().reset_index().drop(['level_1'], axis=1).rename(columns={'level_0':'old_index', 0:'genre'})

In [ ]:
genres['movieId'] = genres.old_index+1

,old_index,genre,movieId
0,0,Adventure,1
1,0,Animation,1
2,0,Children,1
3,0,Comedy,1
4,0,Fantasy,1
5,1,Adventure,2
6,1,Children,2
7,1,Fantasy,2
8,2,Comedy,3
9,2,Romance,3


In [ ]:
genres_wide = pd.get_dummies(genres['genre'])

In [ ]:
genres_wide = genres_wide.join(genres['movieId'])

In [ ]:
movies_full = movies.merge(genres_wide.groupby('movieId').sum().reset_index(), on='movieId')

In [ ]:
# to only get years, not sequels
movies_full['year']= movies_full.title.str.extract('([\(\d\)]+)', expand=False).str.replace('\(','').str.replace('\)','')
# to only remove years, not sequels
movies_full['title_only']=movies_full.title.str.replace('([\(\d\)]+)', '').str.strip()

In [ ]:
genres_list = ['all'].append(movies_full.columns[3:-2])

NameError: name 'movies_full' is not defined

## recommend

## by ratings

In [ ]:
m_rating = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
m_rating['rating_count'] = ratings.groupby('movieId')['rating'].count()

,rating,rating_count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52


In [ ]:
def top_movies(n):
    top = m_rating[m_rating['rating_count']>1].sort_values(['rating', 'rating_count'], ascending=False).head(n)
    top = top.merge(movies, on='movieId')
    return top

## by popularity weighted ratings

In [ ]:
m_rating = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
m_rating['rating_count'] = ratings.groupby('movieId')['rating'].count()
m_rating['weight'] = m_rating['rating_count']/m_rating['rating_count'].sum()
m_rating['weighted_rating'] = m_rating['rating']*m_rating['weight']

,rating,rating_count,weight,weighted_rating
movieId,,,,
1,3.920930,215,0.002132,0.008360
2,3.431818,110,0.001091,0.003744
3,3.259615,52,0.000516,0.001681


In [ ]:
def top_movies_weight(n):
    top = m_rating.sort_values(['weighted_rating'], ascending=False).head(n)
    top = top.merge(movies, on='movieId')
    return top

### by popularity rated ranking for specific genre

In [ ]:
def top_movies_weight_option(option='all', n=5):
    if option == 'all':
        # list of all movies
        option_movies = movies_full['movieId'].to_list()
    else:
        # list of movies from the selected genre by movieId
        option_movies = movies_full.query(f'{option}==1')['movieId'].to_list()
        
    # average ratings and rating count for movies of the same genre
    m_rating_option = pd.DataFrame(ratings.loc[ratings.movieId.isin(option_movies)].groupby('movieId')['rating'].mean())
    m_rating_option['rating_count'] = ratings.loc[ratings.movieId.isin(option_movies)].groupby('movieId')['rating'].count()
    # popularity weights
    m_rating_option['weight'] = m_rating_option['rating_count']/m_rating_option['rating_count'].sum()
    m_rating_option['weighted_rating'] = m_rating_option['rating']*m_rating_option['weight']
    top = m_rating_option.sort_values(['weighted_rating'], ascending=False).head(n)
    top = top.merge(movies, on='movieId')
    return top

### by movie similarity

In [ ]:
user_movie_pivot = pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId')

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# at least 10 recommendations

def recomm_item_based(movie_name, n):
    if len(movies.loc[movies.title.str.contains(movie_name),'movieId'])>1:
        return 'ambigious name'
    else:
        movie = movies.loc[movies.title.str.contains(movie_name),'movieId'].item()
        Id_ratings = user_movie_pivot[movie]
        corr_to_Id = pd.DataFrame(user_movie_pivot.corrwith(Id_ratings), columns=['PearsonR'])
        corr_to_Id.dropna(inplace=True)
        corr_to_Id = corr_to_Id.join(m_rating[['rating', 'rating_count']])
        corr_to_Id.drop(movie, inplace=True) # drop movie itself
        top = corr_to_Id[corr_to_Id['rating_count']>=10].sort_values(['PearsonR', 'rating', 'rating_count'], ascending=False).head(n)
        top = top.merge(movies, left_index=True, right_on="movieId")
        name_movie = movies.loc[movies.movieId==movie]['title'].item()
        print(f'The {n} most similar movies to {name_movie} are:')
        return top

In [ ]:
# at least 10 common recommenders

def recomm_item_based10(movie_name, n):
    if len(movies.loc[movies.title.str.contains(movie_name),'movieId'])>1:
        return 'ambigious name'
    else:
        movie = movies.loc[movies.title.str.contains(movie_name),'movieId'].item()
        common_viewers = user_movie_pivot.loc[user_movie_pivot[movie].notna(), :] # only those viewers that saw the movie
        common_viewers = common_viewers.loc[:,common_viewers.notna().count()>=10] # and only those movies with 10 common viewers
        rest_ratings = common_viewers[movie] # ratings of these viewers
        corr_to_Id = pd.DataFrame(common_viewers.corrwith(rest_ratings), columns=['PearsonR']) # corr with these movies
        corr_to_Id.dropna(inplace=True)
        corr_to_Id = corr_to_Id.join(m_rating[['rating','rating_count']], how='left')
        corr_to_Id.drop(movie, inplace=True) # drop movie itself
        top = corr_to_Id.sort_values(['PearsonR','rating', 'rating_count'], ascending=False).head(n)
        top = top.merge(movies, left_index=True, right_on="movieId")
        name_movie = movies.loc[movies.movieId==movie]['title'].item()
        print(f'The {n} most similar movies to {name_movie} are:')
        return top

### by viewer similarities

In [ ]:
user_movie_null = user_movie_pivot.fillna(0)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
viewer_sim = pd.DataFrame(cosine_similarity(user_movie_null),
                                 columns=user_movie_null.index, 
                                 index=user_movie_null.index)

In [ ]:
def recomm_user_based(viewer,n=5):
    weights = viewer_sim.query('userId!=@viewer')[viewer]/sum(viewer_sim.query('userId!=@viewer')[viewer])
    unseen_movies = user_movie_null.loc[user_movie_null.index!=viewer, user_movie_null.loc[viewer,:]==0]
    weighted_average = pd.DataFrame(unseen_movies.T.dot(weights), columns=['weighted_rating'])
    top = weighted_average.join(m_rating[['rating','rating_count']], how='left')
    top = top.sort_values(['weighted_rating','rating', 'rating_count'], ascending=False).head(n)
    top = top.merge(movies, left_index=True, right_on="movieId")
    print(f'for viewer {viewer} we recommend the following {n} movies based on ratings of viewers with similar taste:')
    return top    